<a id="1."></a> 
# Installing and Importing Libraries

In [1]:
!pip install mediapipe opencv-python

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import os

<a id="2."></a> 
# Implementation of Hand Gesture Recognition System

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

In [4]:
# Function to count opened fingers for both hands
def count_opened_fingers(hand_landmarks):
    
    # Define finger landmark indices
    finger_tip_indices = [4, 8, 12, 16, 20]
    
    # Count opened fingers
    opened_fingers = 0
    
    for finger_tip_index in finger_tip_indices:
        
        # Check if the finger is opened (y-coordinate of finger tip landmark is higher than y-coordinate of its adjacent landmark)
        if hand_landmarks.landmark[finger_tip_index].y < hand_landmarks.landmark[finger_tip_index - 1].y:
            opened_fingers += 1
    
     # Return 0 if no fingers are opened
    if opened_fingers == 0:
        return 0
    else:
        return opened_fingers

In [5]:
# Initialize webcam
cap = cv2.VideoCapture(0)

with mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.5, max_num_hands=2) as hands:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert BGR to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        # Flip image horizontally
        image = cv2.flip(image, 1)
        
        # Set flag
        image.flags.writeable = False
        
        # Detections
        results = hands.process(image)
        
        # Set flag to true
        image.flags.writeable = True

        # RGB 2 BGR
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.multi_hand_landmarks:
            total_opened_fingers = 0
            for hand_landmarks in results.multi_hand_landmarks:
                
                # Draw hand landmarks
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
                # Count opened fingers for each hand
                opened_fingers = count_opened_fingers(hand_landmarks)
                
                # Accumulate total opened fingers
                total_opened_fingers += opened_fingers
                
                # Display count on screen for each hand
                cv2.putText(image, str(opened_fingers),
                            (int(hand_landmarks.landmark[0].x * image.shape[1]),
                             int(hand_landmarks.landmark[0].y * image.shape[0]) + 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (84, 44, 0), 2, cv2.LINE_AA)
        
         # Display total count for both hands
            if total_opened_fingers == 10:
                cv2.putText(image, "10", (570, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (84, 44, 0), 2, cv2.LINE_AA)
            else:
                cv2.putText(image, str(total_opened_fingers), (570, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (84, 44, 0), 2, cv2.LINE_AA)

        # Rendering results
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(image, hand, mp_hands.HAND_CONNECTIONS, 
                                        mp_drawing.DrawingSpec(color=(26, 0, 197),
                                                               thickness=2, circle_radius=4),
                                        mp_drawing.DrawingSpec(color=(246, 246, 253),
                                                               thickness=2, circle_radius=2),
                                         )
        # Display image
        cv2.imshow('Hand Tracking', image)

        # Break loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release webcam and close all windows
cap.release()
# cv2.destroyAllWindows()

KeyboardInterrupt: 